In [37]:
# Pfad zu Chunk File festlegen

#extract = "origin"
extract = "geschichtsbuch"


json_chunk_origin = "docs/chunks_dan_brown.json"
json_chunk_geschichtsbuch = "docs/chunks_geschichtsbuch.json"

json_embedding_geschichtsbuch = "docs/embeddings_geschichtsbuch.json"
json_embedding_origin = "docs/embeddings_dan_brown.json"

chroma_collection_geschichtsbuch = "geschichtsbuch_collection"
chroma_collection_origin = "dan_brown_collection"

chroma_db_geschichtsbuch = "docs/geschichtsbuch_chroma_db"
chroma_db_origin = "docs/dan_brown_chroma_db"


if extract == "geschichtsbuch": 
    json_chunk_path = json_chunk_geschichtsbuch
    json_embedding_path = json_embedding_geschichtsbuch
    chroma_collection = chroma_collection_geschichtsbuch
    chroma_collection_db = chroma_db_geschichtsbuch
else:
    json_chunk_path = json_chunk_origin
    json_embedding_path = json_embedding_origin
    chroma_collection = chroma_collection_origin
    chroma_collection_db = chroma_db_origin

print(f"json_chunk_path is: {json_chunk_path}")
print(f"json_embedding_path is: {json_embedding_path}")
print(f"chroma_collection is: {chroma_collection}")
print(f"chroma_collection_db is: {chroma_collection_db}")

json_chunk_path is: docs/chunks_geschichtsbuch.json
json_embedding_path is: docs/embeddings_geschichtsbuch.json
chroma_collection is: geschichtsbuch_collection
chroma_collection_db is: docs/geschichtsbuch_chroma_db


In [38]:
# Benötigte Bibliotheken importieren
import json
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np

In [39]:
# Chunks aus JSON-Datei laden
with open(json_chunk_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    chunks = data['chunks']

print(f"Anzahl geladener Chunks: {len(chunks)}")
print(f"Beispiel-Chunk: {chunks[0] if chunks else 'Keine Chunks gefunden'}")

Anzahl geladener Chunks: 369
Beispiel-Chunk: {'chunk_id': 0, 'text': '\n\n## Seite 12\n\nVorwort\nVorliegendes Buch baut auf die \'Schweizer Geschichte " von Dr. Ludwig Suter auf, die sich bei der obligatorischen Einführung in den Sekundär-, Bezirks -, Fortbildungs-, Realschulen , Gymnasien und Lehrerseminare vor züglich bewährt hat.\nIm Verlaufe der Schuljahre haben aber Fachund Kennerkreise , ge stützt auf ihre praktischen Erfahrungen, doch den Wunsch geäußert, es möchte manchenorts namentlich für die untern Stufen der Sekundär - , Fort bildungs -und Mittelschulen auf der Grundlage der \'Schweizer Geschichte " von Or. Ludwig Suter ein Lehrmittel geschaffen werden, das inhaltlich und methodisch auf die noch weniger entwickelte Fassungs kraft der Schüler dieser Stufe speziell Rücksicht nimmt. Dem Rate des Autors folgend , betraute die VerlagsanstaltBenziger L Co.A. G.in Einsiedeln mich mit dieser Aufgabe . Ob und wie weit sie gelungen ist , mögen die HH. Lehrer und Fachprofessoren , di

In [40]:
# Modell und Tokenizer laden
model_name = "intfloat/multilingual-e5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Modell in Evaluation-Modus setzen
model.eval()

print(f"Modell {model_name} erfolgreich geladen")

Modell intfloat/multilingual-e5-small erfolgreich geladen


In [41]:
# Funktion zum Erstellen von Embeddings
def get_embeddings(texts, batch_size=32):
    """
    Erstellt Embeddings für eine Liste von Texten
    """
    all_embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        
        # Für E5-Modelle: Text mit "query: " oder "passage: " prefix versehen
        # Hier verwenden wir "passage: " für Dokument-Chunks
        batch_with_prefix = [f"passage: {text}" for text in batch]
        
        # Tokenisierung
        encoded = tokenizer(
            batch_with_prefix,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors='pt'
        )
        
        # Embeddings erstellen (ohne Gradient-Berechnung)
        with torch.no_grad():
            outputs = model(**encoded)
            # Mean pooling über alle Token (außer Padding)
            attention_mask = encoded['attention_mask']
            token_embeddings = outputs.last_hidden_state
            
            # Masked mean pooling
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            embeddings = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
            
            # Normalisierung
            embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
            
            all_embeddings.append(embeddings.cpu().numpy())
        
        if (i + batch_size) % 100 == 0:
            print(f"Verarbeitet: {min(i + batch_size, len(texts))}/{len(texts)} Chunks")
    
    return np.vstack(all_embeddings)

print("Embedding-Funktion definiert")

Embedding-Funktion definiert


In [42]:
# Embeddings für alle Chunks erstellen
print("Starte Embedding-Erstellung...")

# Je nach Chunk-Format anpassen (entweder Liste von Strings oder Liste von Dicts)
if isinstance(chunks[0], dict):
    # Wenn Chunks Dictionaries sind, Text-Feld extrahieren
    chunk_texts = [chunk.get('text', chunk.get('content', str(chunk))) for chunk in chunks]
else:
    # Wenn Chunks bereits Strings sind
    chunk_texts = chunks

embeddings = get_embeddings(chunk_texts)

print(f"\nEmbeddings erstellt!")
print(f"Shape: {embeddings.shape}")
print(f"Embedding-Dimension: {embeddings.shape[1]}")

Starte Embedding-Erstellung...

Embeddings erstellt!
Shape: (369, 384)
Embedding-Dimension: 384


In [43]:
# Embeddings speichern
output_data = []

for i, chunk in enumerate(chunks):
    if isinstance(chunk, dict):
        # Wenn Chunk bereits ein Dictionary ist, Embedding hinzufügen
        chunk_data = chunk.copy()
        chunk_data['embedding'] = embeddings[i].tolist()
        # Stelle sicher, dass Seiteninformationen enthalten sind
        if 'pages' not in chunk_data:
            chunk_data['pages'] = None
    else:
        # Wenn Chunk ein String ist, Dictionary erstellen
        chunk_data = {
            'text': chunk,
            'embedding': embeddings[i].tolist(),
            'pages': None
        }
    output_data.append(chunk_data)

# In JSON-Datei speichern
with open(json_embedding_path, 'w', encoding='utf-8') as f:

    json.dump(output_data, f, ensure_ascii=False, indent=2)
    print(f"Beispiel (mit Seiteninformation): Chunk 0 -> Seite(n): {output_data[0].get('pages', 'N/A')}")

print(f"Anzahl gespeicherter Embeddings: {len(output_data)}")
print(f"Embeddings gespeichert in: {json_embedding_path}")

Beispiel (mit Seiteninformation): Chunk 0 -> Seite(n): [12]
Anzahl gespeicherter Embeddings: 369
Embeddings gespeichert in: docs/embeddings_geschichtsbuch.json


In [44]:
# ChromaDB importieren
import chromadb
from chromadb.config import Settings

print("ChromaDB importiert")

ChromaDB importiert


In [45]:
# ChromaDB Client und Collection erstellen
# PersistentClient speichert die Daten dauerhaft
chroma_client = chromadb.PersistentClient(path=chroma_collection_db)

# Collection erstellen oder abrufen
# Der Name sollte beschreibend sein
collection_name = chroma_collection
collection = chroma_client.get_or_create_collection(
    name=collection_name,
    metadata={"description": f"Embeddings für {extract} mit multilingual-e5-small"}
)

print(f"ChromaDB Collection '{collection_name}' erstellt/geladen")
print(f"Anzahl bestehender Dokumente in Collection: {collection.count()}")

ChromaDB Collection 'geschichtsbuch_collection' erstellt/geladen
Anzahl bestehender Dokumente in Collection: 369


In [46]:
# Daten für ChromaDB vorbereiten
ids = []
documents = []
embeddings_list = []
metadatas = []

for i, chunk in enumerate(chunks):
    # ID für jedes Dokument (muss eindeutig sein)
    doc_id = f"chunk_{i}"
    ids.append(doc_id)
    
    # Text extrahieren
    if isinstance(chunk, dict):
        text = chunk.get('text', chunk.get('content', str(chunk)))
        # Optionale Metadaten aus dem Chunk extrahieren
        metadata = {k: v for k, v in chunk.items() if k not in ['text', 'content', 'embedding']}
        metadata['chunk_index'] = i
        
        # Explizit Seiteninformationen hinzufügen
        if 'pages' in chunk and chunk['pages']:
            # Konvertiere Liste zu String für ChromaDB-Kompatibilität
            metadata['pages'] = str(chunk['pages'])
            metadata['page_numbers'] = ', '.join(map(str, chunk['pages']))
        else:
            metadata['pages'] = None
            metadata['page_numbers'] = 'N/A'
    else:
        text = chunk
        metadata = {
            'chunk_index': i,
            'pages': None,
            'page_numbers': 'N/A'
        }
    
    documents.append(text)
    embeddings_list.append(embeddings[i].tolist())
    metadatas.append(metadata)

print(f"Daten vorbereitet: {len(ids)} Dokumente")
print(f"Beispiel Metadaten: {metadatas[0]}")

Daten vorbereitet: 369 Dokumente
Beispiel Metadaten: {'chunk_id': 0, 'length': 1500, 'pages': '[12]', 'start_pos': 0, 'end_pos': 1500, 'chunk_index': 0, 'page_numbers': '12'}


In [47]:
# In ChromaDB speichern
# Collection löschen und neu erstellen für frische Daten
if collection.count() > 0:
    print(f"⚠️  Collection enthält bereits {collection.count()} Dokumente.")
    print(f"Lösche bestehende Collection '{collection_name}'...")
    chroma_client.delete_collection(name=collection_name)
    print("✓ Collection gelöscht.")
    
    # Collection neu erstellen
    collection = chroma_client.get_or_create_collection(
        name=collection_name,
        metadata={"description": f"Embeddings für {extract} mit multilingual-e5-small"}
    )
    print("✓ Collection neu erstellt.")

# Neue Daten hinzufügen (jetzt in leere Collection)
collection.add(
    ids=ids,
    documents=documents,
    embeddings=embeddings_list,
    metadatas=metadatas
)

print(f"\n✓ Embeddings in ChromaDB gespeichert!")
print(f"  Collection: {collection_name}")
print(f"  Anzahl Dokumente: {collection.count()}")

⚠️  Collection enthält bereits 369 Dokumente.
Lösche bestehende Collection 'geschichtsbuch_collection'...
✓ Collection gelöscht.
✓ Collection neu erstellt.

✓ Embeddings in ChromaDB gespeichert!
  Collection: geschichtsbuch_collection
  Anzahl Dokumente: 369


## Test: Ähnlichkeitssuche mit ChromaDB

Teste die Suche in der ChromaDB mit einer Beispiel-Query:

In [48]:
# Beispiel-Query für Ähnlichkeitssuche
query_text = "Wer war Wilhelm Tell?"  # Passen Sie die Query an Ihre Daten an

# Query-Embedding erstellen (mit "query:" Prefix für E5)
query_with_prefix = f"query: {query_text}"
query_encoded = tokenizer(
    [query_with_prefix],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors='pt'
)

with torch.no_grad():
    query_outputs = model(**query_encoded)
    attention_mask = query_encoded['attention_mask']
    token_embeddings = query_outputs.last_hidden_state
    
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    query_embedding = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    query_embedding = torch.nn.functional.normalize(query_embedding, p=2, dim=1)
    query_embedding = query_embedding.cpu().numpy()[0]

# Suche in ChromaDB
results = collection.query(
    query_embeddings=[query_embedding.tolist()],
    n_results=3
)

print(f"Query: '{query_text}'\n")
print("Top 3 ähnlichste Chunks:")
for i, (doc, distance, metadata) in enumerate(zip(results['documents'][0], results['distances'][0], results['metadatas'][0])):
    print(f"\n{i+1}. (Distanz: {distance:.4f})")
    pages_info = metadata.get('page_numbers', 'N/A')
    print(f"   Seite(n): {pages_info}")
    print(f"   Chunk Index: {metadata.get('chunk_index', 'N/A')}")
    print(f"   Text: {doc[:200]}...")  # Nur erste 200 Zeichen

Query: 'Wer war Wilhelm Tell?'

Top 3 ähnlichste Chunks:

1. (Distanz: 0.3789)
   Seite(n): 59
   Chunk Index: 68
   Text:  Torwächter und Kriegsknechte und steckten die Burg in Brand . Der Vogt wurde gefangen ge nommen und mußte auf der Landesgrenze den Eid schwören , das Land nie mehr betreten zu wollen. - Zu gleicher Z...

2. (Distanz: 0.4116)
   Seite(n): 57, 58, 59
   Chunk Index: 67
   Text:  Apfel vom Kopfe zu schießen. Teil bat , man möge ihm die Strafe erlassen . Geßler aber drohte, erwerbe ihn mit dem Kinde töten lassen . Da betete Tell zu Gott , steckte noch einen zweiten Pfeil in de...

3. (Distanz: 0.4183)
   Seite(n): 57
   Chunk Index: 66
   Text: , wie sie das fremde Joch abschütteln können . Stauffacher begab sich alsbald nach Uri zu Walter Fürst, einem edeln , erfahrenen Manne , und traf dort auch den flüchtigen Arnold AnderHalden . Sie besc...
